# Rollout

In [6]:
import torch
import gymnasium as gym
import numpy as np
import mediapy as media
# 필요한 클래스 및 함수 임포트
from cleanrl.cleanrl.ppo_continuous_action import Agent, Args, ppo_make_env
import cv2
from robosuite.utils.camera_utils import CameraMover

visualize = True
frames = []

# Argument 설정
args = Args()
task_id = 'pickplace'
seed = 0
gamma = 0.99
num_episodes = 1
render_camera = ['birdview','agentview'] #('frontview', 'birdview', 'agentview', 'robot0_robotview', 'robot0_eye_in_hand')
camera_names = render_camera


# 환경 생성
env = gym.vector.SyncVectorEnv(
    [ppo_make_env(
        task_id=task_id, 
        reward_shaping=args.reward_shaping,
        idx=0, 
        control_mode="OSC_POSITION",
        capture_video=False, 
        run_name="eval", 
        gamma= args.gamma, 
        active_rewards="r",
        active_image=True, 
        fix_object=args.fix_object,
        wandb_enabled=False,
        verbose=True,
        control_freq=20,
        render_camera=render_camera,
        camera_names=camera_names,

        )
    ]
)

#cameramover = CameraMover(env.envs[0], camera=render_camera, init_camera_pos=None, init_camera_quat=None)
#print(env.envs[0].sim.model.cam_pos[camera_id])
#print(env.envs[0].sim.model.cam_quat[camera_id])

#env.envs[0].camera_pos = np.array([-0.2, 0.0, 0.5])
#env.envs[0].camera_quat = np.array([0.70710678,0.,0.,0.70710678])
#env.envs[0].camera_id = camera_id

#env.envs[0].sim.model.cam_pos[camera_id] = np.array([0.0, 0.0, 0.0])  # 높이 2.0으로 설정
#env.envs[0].sim.model.cam_quat[camera_id] = np.array([1.0, 0.0, 0.0, 0.0])  # 카메라의 회전 설정


def colorize_depth(frame):
    # Assuming the depth image is in float32 and contains values representing distances.
    # Normalize the depth image to 0-255 for visualization
    min_depth = np.min(frame)
    max_depth = np.max(frame)
    normalized_depth = 255 * (frame - min_depth) / (max_depth - min_depth)
    normalized_depth = normalized_depth.astype(np.uint8)

    # Apply a colormap for better visualization (COLORMAP_JET is commonly used)
    colorized_depth = cv2.applyColorMap(normalized_depth, cv2.COLORMAP_JET)
    return colorized_depth


# 디바이스 설정 (cuda가 가능하면 cuda 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device("cuda"):
    print("Using CUDA")
else :
    assert device == torch.device("cpu")

# Agent 초기화 및 모델 불러오기
agent = Agent(env).to(device)
agent.eval()  # 평가 모드로 전환

# 평가 수행
total_rewards = []
viewer_image_key = 'birdview'+'_depth'
for episode in range(num_episodes):
    obs, _ = env.reset(seed=args.seed)
    #env.envs[0].sim.model.cam_pos[camera_id] = [0.1, -0.25, 1.5]
    #env.envs[0].sim.model.cam_quat[camera_id] = [0.70710678,0.,0.,0.70710678]

    #env.step([0, 0, 0, 0])

    obs = torch.Tensor(obs).to(device)
    done = False
    episode_reward = 0
    
    image_frame = env.envs[0].image_states[viewer_image_key]
    if not viewer_image_key.endswith('depth'):
        image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # numpy 배열로 변환
    else:
        image_frame = np.array(image_frame[::-1, :, :], dtype=np.float32)
    image_frame = colorize_depth(image_frame)
    frames.append(image_frame)
    # convert image_frame cv2 image
    #image_frame = cv2.cvtColor(image_frame, cv2.COLOR_RGB2BGR)    
    for i in range(30):

        obs = torch.Tensor(obs).to(device)
        action = env.envs[0].action_space.sample() 


        #action[0] = 0
        #action[1] = 0
        #action[2] = -0.1
        #action[3] = 0.
        obs, reward, terminations, truncations, infos = env.step([action])
        done = np.logical_or(terminations, truncations).any()
        episode_reward += reward[0]  # 첫 번째 환경의 보상 합산
        
        # 새로운 프레임 가져오기 및 변환
        image_frame = env.envs[0].image_states[viewer_image_key]
        if not viewer_image_key.endswith('depth'):
            image_frame = np.array(image_frame[::-1, :, :], dtype=np.uint8)  # numpy 배열로 변환
        else:
            image_frame = np.array(image_frame[::-1, :, :], dtype=np.float32)

        if viewer_image_key.endswith('depth'):
           image_frame = colorize_depth(image_frame)



        #image_frame = cv2.cvtColor(image_frame, cv2.COLOR_RGB2BGR)
        # draw text on image_frame episode reward, reward, small text
        #cv2.putText(image_frame, f"Episode Reward: {episode_reward:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        #cv2.putText(image_frame, f"Reward: {reward[0]:.2f}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        # draw action on image_frame at bottom
        cv2.putText(image_frame, f"Action: {action}", (10, 190), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 2)
        
        frames.append(image_frame)
        

    print(f"Episode {episode + 1}: Total Reward: {episode_reward}")
    total_rewards.append(episode_reward)

    env.close()

    # 평균 리턴 출력
    avg_reward = np.mean(total_rewards)
    print(f"Average Reward over {num_episodes} episodes: {avg_reward}")
    
    
    media.show_video(frames, fps=30)



init_env: task_id: pickplace
### controller_config: OSC_POSITION ###
#### J PickPlace ####
fix_object:False
start with grasp lock: True
control_freq: 20
ignore_done: False
########################
### Observation keys ###
Key: robot0_eef_pos, size: 3
Key: robot0_eef_quat, size: 4
Key: robot0_eef_vel_lin, size: 3
Key: robot0_eef_vel_ang, size: 3
Key: robot0_gripper_qpos, size: 6
Key: robot0_gripper_qvel, size: 6
Key: Can_pos, size: 3
Key: Can_quat, size: 4
Key: Can_to_robot0_eef_pos, size: 3
Key: Can_to_robot0_eef_quat, size: 4
Key: robot0_proprio-state, size: 25
Key: object-state, size: 14
Total observation size: 78
########################
####### Options ########
task_id: pickplace
active_rewards: r
control_mode: OSC_POSITION
reward_shaping: False
fix_object: False
active_image: True
wandb_enabled: False
########################
Episode 1: Total Reward: 0.0
Average Reward over 1 episodes: 0.0


This browser does not support the video tag.


In [27]:
from my_utils import init_env

env = init_env(
    task_id='pickplace',
    reward_shaping=True,
    control_mode="OSC_POSITION",
    active_rewards="r",
    active_image=True,
    fix_object=False,
    wandb_enabled=False,
    verbose=True,
    control_freq=20,
    render_camera='birdview',
    camera_names='birdview',
)



init_env: task_id: pickplace
### controller_config: OSC_POSITION ###
#### J PickPlace ####
fix_object:False
start with grasp lock: True
control_freq: 20
ignore_done: True
########################
### Observation keys ###
Key: robot0_eef_pos, size: 3
Key: robot0_eef_quat, size: 4
Key: robot0_eef_vel_lin, size: 3
Key: robot0_eef_vel_ang, size: 3
Key: robot0_gripper_qpos, size: 6
Key: robot0_gripper_qvel, size: 6
Key: Can_pos, size: 3
Key: Can_quat, size: 4
Key: Can_to_robot0_eef_pos, size: 3
Key: Can_to_robot0_eef_quat, size: 4
Key: robot0_proprio-state, size: 25
Key: object-state, size: 14
Total observation size: 78
########################
####### Options ########
task_id: pickplace
active_rewards: r
control_mode: OSC_POSITION
reward_shaping: True
fix_object: False
active_image: True
wandb_enabled: False
########################


In [34]:

birdview_cam_id = env.sim.model.camera_name2id("agentview")  # 예시로 'agentview' 카메라를 사용

In [4]:
action = env.envs[0].action_space.sample()    
print(action.shape)
action = np.array([0.1, 0.1, 0.1, 0.1])
print(action.shape)

(4,)
(4,)
